## Mapping U.S. Race Demographics
Mapping 2017 Census race demographics by zip code (33,000+)

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

---

Zip Codes & Cities:

In [2]:
# All US zip codes (from: https://simplemaps.com/data/us-zips)
zip_city = pd.read_csv('uszips.csv')

In [3]:
zip_city.head(1)

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,all_county_weights,imprecise,military,timezone
0,601,18.18004,-66.75218,Adjuntas,PR,Puerto Rico,True,NaN,18570,111.4,72001,Adjuntas,"{'72001':99.43,'72141':0.57}",False,False,America/Puerto_Rico


In [4]:
zip_city.zip.dtype

dtype('int64')

In [5]:
len(zip_city)

33099

In [6]:
zip_city = zip_city[['zip', 'city', 'state_name', 'population', 'density']]

In [7]:
zip_city.head(1)

,zip,city,state_name,population,density
0,601,Adjuntas,Puerto Rico,18570,111.4


---

Zip Codes Shapefile:

In [8]:
# Shapefile of all US ZCTA boundaries https://www.census.gov/programs-surveys/geography/guidance/geo-areas/zctas.html
zip_shape = gpd.read_file('tl_2019_us_zcta510.shp')

In [9]:
len(zip_shape)

33144

In [10]:
zip_shape.head(1)

,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
0,43451,43451,B5,G6350,S,63484186,157689,+41.3183010,-083.6174935,"POLYGON ((-83.70873 41.32733, -83.70815 41.327..."


In [11]:
zip_shape.dtypes

ZCTA5CE10       object
GEOID10         object
CLASSFP10       object
MTFCC10         object
FUNCSTAT10      object
ALAND10          int64
AWATER10         int64
INTPTLAT10      object
INTPTLON10      object
geometry      geometry
dtype: object

In [12]:
# Converting to enable comparison of zip code values in 'zip_race' below
zip_shape['GEOID10'] = zip_shape['GEOID10'].astype(int)

---

Zip Codes & Race Demographics:

In [13]:
# https://factfinder.census.gov/faces/tableservices/jsf/pages/productview.xhtml?src=bkmk
zip_race = pd.read_csv('census_race.csv')

In [14]:
len(zip_race)

33120

In [15]:
zip_race.head(1)

,Id,Id2,Geography,Estimate; Total:,Margin of Error; Total:,Estimate; Not Hispanic or Latino:,Margin of Error; Not Hispanic or Latino:,Estimate; Not Hispanic or Latino: - White alone,Margin of Error; Not Hispanic or Latino: - White alone,Estimate; Not Hispanic or Latino: - Black or African American alone,...,Estimate; Hispanic or Latino: - Native Hawaiian and Other Pacific Islander alone,Margin of Error; Hispanic or Latino: - Native Hawaiian and Other Pacific Islander alone,Estimate; Hispanic or Latino: - Some other race alone,Margin of Error; Hispanic or Latino: - Some other race alone,Estimate; Hispanic or Latino: - Two or more races:,Margin of Error; Hispanic or Latino: - Two or more races:,Estimate; Hispanic or Latino: - Two or more races: - Two races including Some other race,Margin of Error; Hispanic or Latino: - Two or more races: - Two races including Some other race,"Estimate; Hispanic or Latino: - Two or more races: - Two races excluding Some other race, and three or more races","Margin of Error; Hispanic or Latino: - Two or more races: - Two races excluding Some other race, and three or more races"
0,8600000US00601,601,ZCTA5 00601,17599,261,66,48,59,47,0,...,0,21,3641,588,135,153,21,24,114,147


In [16]:
zip_race.columns

Index(['Id', 'Id2', 'Geography', 'Estimate; Total:', 'Margin of Error; Total:',
       'Estimate; Not Hispanic or Latino:',
       'Margin of Error; Not Hispanic or Latino:',
       'Estimate; Not Hispanic or Latino: - White alone',
       'Margin of Error; Not Hispanic or Latino: - White alone',
       'Estimate; Not Hispanic or Latino: - Black or African American alone',
       'Margin of Error; Not Hispanic or Latino: - Black or African American alone',
       'Estimate; Not Hispanic or Latino: - American Indian and Alaska Native alone',
       'Margin of Error; Not Hispanic or Latino: - American Indian and Alaska Native alone',
       'Estimate; Not Hispanic or Latino: - Asian alone',
       'Margin of Error; Not Hispanic or Latino: - Asian alone',
       'Estimate; Not Hispanic or Latino: - Native Hawaiian and Other Pacific Islander alone',
       'Margin of Error; Not Hispanic or Latino: - Native Hawaiian and Other Pacific Islander alone',
       'Estimate; Not Hispanic or Latino

In [17]:
# Keeping relevant columns only
columns = ['Id2', 
           'Estimate; Total:',
           'Estimate; Not Hispanic or Latino: - White alone',
           'Estimate; Not Hispanic or Latino: - Black or African American alone',
           'Estimate; Not Hispanic or Latino: - American Indian and Alaska Native alone',
           'Estimate; Not Hispanic or Latino: - Asian alone',
           'Estimate; Not Hispanic or Latino: - Native Hawaiian and Other Pacific Islander alone',
           'Estimate; Not Hispanic or Latino: - Some other race alone',
           'Estimate; Not Hispanic or Latino: - Two or more races:',
           'Estimate; Not Hispanic or Latino: - Two or more races: - Two races including Some other race',
           'Estimate; Not Hispanic or Latino: - Two or more races: - Two races excluding Some other race, and three or more races',
           'Estimate; Hispanic or Latino:']

In [18]:
zip_race = zip_race[columns]

In [19]:
# Simplifying column names
zip_race = zip_race.rename({'Id2':'Zip_Code', 
                     'Estimate; Total:':'Population', 
                     'Estimate; Not Hispanic or Latino: - White alone':'White', 
                     'Estimate; Not Hispanic or Latino: - Black or African American alone':'Black', 
                     'Estimate; Not Hispanic or Latino: - American Indian and Alaska Native alone':'Native_American', 
                     'Estimate; Not Hispanic or Latino: - Asian alone':'Asian', 
                     'Estimate; Not Hispanic or Latino: - Native Hawaiian and Other Pacific Islander alone':'Hawaiian_Pacific_Islander',
                     'Estimate; Not Hispanic or Latino: - Some other race alone':'Other', 
                     'Estimate; Not Hispanic or Latino: - Two or more races:':'Mixed_Race1', 
                     'Estimate; Not Hispanic or Latino: - Two or more races: - Two races including Some other race':'Mixed_Race2',
                     'Estimate; Not Hispanic or Latino: - Two or more races: - Two races excluding Some other race, and three or more races':'Mixed_Race3',
                     'Estimate; Hispanic or Latino:':'Latino'}, axis=1)

In [20]:
zip_race['Mixed_Race'] = zip_race['Mixed_Race1'] + zip_race['Mixed_Race2'] + zip_race['Mixed_Race3']
zip_race = zip_race.drop(['Mixed_Race1', 'Mixed_Race2', 'Mixed_Race3'], axis=1)

In [21]:
zip_race.head(1)

,Zip_Code,Population,White,Black,Native_American,Asian,Hawaiian_Pacific_Islander,Other,Latino,Mixed_Race
0,601,17599,59,0,7,0,0,0,17533,0


In [22]:
zip_race.to_csv('raw_race_data.csv', index=False)

---

#### Checking Population Data Accuracy

In [22]:
# Number of records where total population doesn't match sum of individual race population figures
counter = 0
for i, row in zip_race.iterrows():
    vals = row.values[2:]
    total = sum(vals)
    if total != row.Population:
        counter += 1
print(counter)

25376


In [23]:
# Checking how many rows are off by more than 10%
counter = 0
for i, row in zip_race.iterrows():
    vals = row.values[2:]
    total = sum(vals)
    if total > (row.Population + row.Population * .1): 
        counter += 1
print(counter)

606


*About 2% of records contain demographic data that are off by more than 10%, which is a low enough to be acceptable

---

#### Normalizing Population Data
(to account for slightly inaccurate population data)

*Separating records w/ missing population data (b/c will result in nulls when normalizing):

In [24]:
len(zip_race[zip_race.Population == 0])

317

In [25]:
nulls_df = zip_race[zip_race.Population == 0]
not_nulls_df = zip_race[zip_race.Population != 0]

In [26]:
not_normalized = not_nulls_df.loc[:,:'Population']
normalized = not_nulls_df.loc[:, 'White':]
normalized = normalized.div(normalized.sum(axis=1), axis=0).round(2) * 100  # rounding to nearest whole number for simpler presentation of data in map
joined = pd.concat([not_normalized, normalized], axis=1)

In [27]:
cleaned = pd.concat([joined, nulls_df]).sort_values('Zip_Code')

In [28]:
cleaned.head(2)

,Zip_Code,Population,White,Black,Native_American,Asian,Hawaiian_Pacific_Islander,Other,Latino,Mixed_Race
0,601,17599,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0
1,602,39209,4.0,0.0,0.0,0.0,0.0,0.0,92.0,4.0


In [29]:
cleaned.isnull().sum()

Zip_Code                     0
Population                   0
White                        0
Black                        0
Native_American              0
Asian                        0
Hawaiian_Pacific_Islander    0
Other                        0
Latino                       0
Mixed_Race                   0
dtype: int64

---

#### Linking City Name to Zip Code

In [30]:
cleaned.head(1)

,Zip_Code,Population,White,Black,Native_American,Asian,Hawaiian_Pacific_Islander,Other,Latino,Mixed_Race
0,601,17599,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0


In [31]:
zip_city.head(1)

,zip,city,state_name,population,density
0,601,Adjuntas,Puerto Rico,18570,111.4


*Note that population is lower in 'cleaned' because this data is from 2017, whereas zip_city is from 2019; I relied on 'cleaned' population to be consistent w/ racial demographics, though I did include zip_city density in case it's useful

In [32]:
cities = []
states = []
densities = []
for i, row in cleaned.iterrows():
    try:
        city = zip_city['city'][zip_city['zip'] == row['Zip_Code']].iloc[0]
        state = zip_city['state_name'][zip_city['zip'] == row['Zip_Code']].iloc[0]
        density = zip_city['density'][zip_city['zip'] == row['Zip_Code']].iloc[0]
    except:
        city = np.nan
        state = np.nan
        density = np.nan
    cities.append(city)
    states.append(state)
    densities.append(density)

In [33]:
# Alternative method
"""cities = []
states = []
densities = []
for i, row in zip_race.iterrows():
    for col in ['city', 'state_name', 'density']:
        lists = [cities, states, densities]
        counter = 0
        try:
            val = zip_city[col][zip_city['zip'] == row['Zip_Code']].iloc[0]
        except:
            val = np.nan
        lists[counter].append(val)
        counter += 1"""

"cities = []\nstates = []\ndensities = []\nfor i, row in zip_race.iterrows():\n    for col in ['city', 'state_name', 'density']:\n        lists = [cities, states, densities]\n        counter = 0\n        try:\n            val = zip_city[col][zip_city['zip'] == row['Zip_Code']].iloc[0]\n        except:\n            val = np.nan\n        lists[counter].append(val)\n        counter += 1"

In [34]:
cleaned['City'] = cities
cleaned['State'] = states
cleaned['Density'] = densities

In [35]:
cols = ['Zip_Code', 'City', 'State', 'Population', 'Density'] + list(cleaned.loc[:, 'White':'Mixed_Race'].columns)
cols

['Zip_Code',
 'City',
 'State',
 'Population',
 'Density',
 'White',
 'Black',
 'Native_American',
 'Asian',
 'Hawaiian_Pacific_Islander',
 'Other',
 'Latino',
 'Mixed_Race']

In [36]:
cleaned[cols].head(1)

,Zip_Code,City,State,Population,Density,White,Black,Native_American,Asian,Hawaiian_Pacific_Islander,Other,Latino,Mixed_Race
0,601,Adjuntas,Puerto Rico,17599,111.4,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0


In [37]:
cleaned = cleaned[cols]

In [38]:
# Converting to str so that values match in QGIS editing (since shapefile zip code values are strings)
cleaned['Zip_Code'] = cleaned['Zip_Code'].astype(str).apply(lambda x:x.zfill(5))

In [39]:
cleaned.head(1)

,Zip_Code,City,State,Population,Density,White,Black,Native_American,Asian,Hawaiian_Pacific_Islander,Other,Latino,Mixed_Race
0,00601,Adjuntas,Puerto Rico,17599,111.4,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0


In [40]:
cleaned.columns[5:]

Index(['White', 'Black', 'Native_American', 'Asian',
       'Hawaiian_Pacific_Islander', 'Other', 'Latino', 'Mixed_Race'],
      dtype='object')

In [41]:
for col in cleaned.columns[5:]:
    cleaned[col] = cleaned[col].astype(int)

In [42]:
cleaned.dtypes

Zip_Code                      object
City                          object
State                         object
Population                     int64
Density                      float64
White                          int32
Black                          int32
Native_American                int32
Asian                          int32
Hawaiian_Pacific_Islander      int32
Other                          int32
Latino                         int32
Mixed_Race                     int32
dtype: object

In [43]:
cleaned.head(1)

,Zip_Code,City,State,Population,Density,White,Black,Native_American,Asian,Hawaiian_Pacific_Islander,Other,Latino,Mixed_Race
0,00601,Adjuntas,Puerto Rico,17599,111.4,0,0,0,0,0,0,100,0


In [44]:
cleaned.to_csv('census_race_cleaned.csv', index=False)

---

#### Delete Records Missing Zip Codes in QGIS
(they correspond to U.S. territories or very small areas)

In [45]:
print(len(zip_shape))
print(len(cleaned))
print(len(zip_city))

33144
33120
33099


In [47]:
missing = []
cleaned_zips = cleaned['Zip_Code'].astype(int).values
for zipc in zip_shape.sort_values('GEOID10')['GEOID10']:
    if zipc not in cleaned_zips or zipc not in zip_city['zip'].values:
        missing.append(zipc)
        
print(len(missing))
print(missing)

45
[801, 802, 820, 823, 824, 830, 831, 840, 841, 850, 851, 17270, 17767, 19542, 20307, 42084, 42731, 45145, 45418, 48921, 54010, 54934, 56177, 63464, 64192, 66019, 83601, 84144, 95250, 95314, 96799, 96910, 96913, 96915, 96916, 96917, 96921, 96928, 96929, 96932, 96950, 96951, 96952, 98205, 98929]


---

In [54]:
cleaned[cleaned.Other > 10]

,Zip_Code,City,State,Population,Density,White,Black,Native_American,Asian,Hawaiian_Pacific_Islander,Other,Latino,Mixed_Race
575,02538,East Wareham,Massachusetts,3571,306.4,74,2,0,2,0,11,3,8
584,02558,Onset,Massachusetts,749,677.0,72,4,0,0,0,24,0,0
629,02672,West Hyannisport,Massachusetts,368,140.6,89,0,0,0,0,11,0,0
2332,08073,Rancocas,New Jersey,248,818.0,77,0,0,0,0,23,0,0
2897,11419,South Richmond Hill,New York,52392,16283.6,4,16,1,28,0,23,19,10
2898,11420,South Ozone Park,New York,49396,8243.5,6,22,1,27,0,11,22,11
3281,12454,Maplecrest,New York,299,10.3,89,0,0,0,0,11,0,0
3290,12464,Phoenicia,New York,790,8.9,85,0,0,0,0,15,0,0
6714,21778,Rocky Ridge,Maryland,1364,24.1,78,0,0,0,0,22,1,0
7374,24082,Critz,Virginia,441,21.6,59,25,0,0,0,15,0,0


*Exclude Other since it's hardly represented